In [1]:
import sys

sys.path.insert(0, "..")


In [2]:
from dataset import LesionDataset

dataset = LesionDataset('data') #local path to data
for patient in dataset.patients():
    """Patient is an instance of the Patient class

    The `Patient` class exposes a number of methods for accessing a patient's associated data:
    - `Patient.image_data(image_type)` - returns the image, prostate mask, and lesion arrays
    corresponding to `image_type`
    - `Patient.world_matrix()` - returns the world matrix of `image_type`
    - `Patient.spacing(image_type)` - returns the voxel spacing of `image_type`
    - `Patient.bvals()` - returns the b-values of the DWI images
    - `Patient.slice_data(image_type, slice_index, crop_to_mask=False, pad=0)` - returns a particular
    slice of the image/mask data, optionally cropped to the prostate mask for easier visualization.
    """
    print(patient.patient_id)

print(dataset.query('ProstateX-0000'))

ProstateX-0000
ProstateX-0001
ProstateX-0002
ProstateX-0003
ProstateX-0004
ProstateX-0005
ProstateX-0006
ProstateX-0007
ProstateX-0008
ProstateX-0009
ProstateX-0010
ProstateX-0011
ProstateX-0012
ProstateX-0013
ProstateX-0014
ProstateX-0015
ProstateX-0016
ProstateX-0017
ProstateX-0018
ProstateX-0019
ProstateX-0020
ProstateX-0021
ProstateX-0022
ProstateX-0023
ProstateX-0024
ProstateX-0025
ProstateX-0026
ProstateX-0027
ProstateX-0028
ProstateX-0029
ProstateX-0030
ProstateX-0031
ProstateX-0032
ProstateX-0033
ProstateX-0034
ProstateX-0035
ProstateX-0036
ProstateX-0037
ProstateX-0038
ProstateX-0039
ProstateX-0040
ProstateX-0041
ProstateX-0042
ProstateX-0043
ProstateX-0044
ProstateX-0045
ProstateX-0046
ProstateX-0047
ProstateX-0048
ProstateX-0049
ProstateX-0050
ProstateX-0051
ProstateX-0053
ProstateX-0054
ProstateX-0055
ProstateX-0057
ProstateX-0058
ProstateX-0059
ProstateX-0060
ProstateX-0061
ProstateX-0062
ProstateX-0063
ProstateX-0064
ProstateX-0065
ProstateX-0066
ProstateX-0067
ProstateX-

In [ ]:
# start setting up classes for resnet50

import torch
import torch.nn as nn
import torch.nn.functional as F

class ResNet50(nn.Module):
    def __init__(self, num_classes=1000):

        # Standard ResNet50 architecture

        super(ResNet50, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 64, 3)
        self.layer2 = self._make_layer(256, 128, 4, stride=2)
        self.layer3 = self._make_layer(512, 256, 6, stride=2)
        self.layer4 = self._make_layer(1024, 512, 3, stride=2)

        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(2048, num_classes)


        self._initialize_weights()

    def _make_layer(self, in_planes, out_planes, blocks, stride=1):
        layers = []
        layers.append(nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False))
        layers.append(nn.BatchNorm2d(out_planes))
        layers.append(nn.ReLU(inplace=True))
        for i in range(1, blocks):
            layers.append(nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False))
            layers.append(nn.BatchNorm2d(out_planes))
            layers.append(nn.ReLU(inplace=True))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

model = ResNet50()
print(model)